# Common Python Data Structures (Guide)

[https://realpython.com/python-data-structures/](https://realpython.com/python-data-structures/)

Neste tutorial, você aprenderá:

- Quais tipos de dados abstratos comuns são integrados à biblioteca padrão do Python
- Como os tipos de dados abstratos mais comuns são mapeados para o esquema de nomenclatura do Python
- Como colocar tipos de dados abstratos em uso prático em vários algoritmos

## Registros, Estuturas e Objetos de Transferência de Dados

[https://realpython.com/python-data-structures/#records-structs-and-data-transfer-objects](https://realpython.com/python-data-structures/#records-structs-and-data-transfer-objects)

Em comparação com os arrays, as estruturas de dados de registro (*record*) fornecem um número fixo de campos. Cada campo pode ter um nome e também pode ter um tipo diferente.

Nesta seção, você verá como implementar registros, estruturas(*struct*) e objetos de dados comuns no Python, usando apenas tipos de dados built-in e classes da biblioteca padrão.

> Nota: Aqui, vamos usar uma definição vaga de registro. Por exemplo, também
> vamos discutir tipos built-in como a tupla, que pode ou não ser considerada
> registros em um sentido estrito, já que não fornecem campos nomeados.

O Python oferece vários tipos de dados que você pode usar para implementar registros, estruturas e objetos de transferência de dados (*data transfer objects*). Nesta seção, você dará uma rápida olhada em cada implementação e suas características únicas. No final, você encontrará um resumo e um guia de decisão que ajudará você a fazer suas próprias escolhas.

### `dict`: Simple Data Objects

Como mencionado anteriormente, os dicionários do Python armazenam um número arbitrário de objetos, cada um identificado por uma chave exclusiva. Os dicionários também são frequentemente chamados de mapas ou matrizes associativas e permitem a pesquisa, a inserção e a exclusão eficientes de qualquer objeto associado a uma determinada chave.

Em Python, é possível usar dicionários como registro ou objeto de dados. Os dicionários são fáceis de criar, pois eles têm seu próprio açúcar sintático embutido na linguagem sob a forma de literais de dicionário. A sintaxe do dicionário é concisa e bastante conveniente para digitar.

Os objetos de dados criados usando dicionários são mutáveis, e há pouca proteção contra nomes de campo com erros de digitação, assim como campos podem ser adicionados e removidos livremente a qualquer momento. Ambas as propriedades podem introduzir bugs surpreendentes, e há sempre uma trade-off a ser feita entre conveniência e resiliência a erro.

In [1]:
car1 = {
    "color": "red",
    "mileage": 3812.4,
    "automatic": True,
}

car2 = {
    "color": "blue",
    "mileage": 40231,
    "automatic": False,
}

In [2]:
# car2.__repr__()
car2

{'color': 'blue', 'mileage': 40231, 'automatic': False}

In [3]:
# Get mileage
car2["mileage"]

40231

In [4]:
# Dicts são mutáveis
car2["mileage"] = 12
car2["windshield"] = "broken"
car2

{'color': 'blue', 'mileage': 12, 'automatic': False, 'windshield': 'broken'}

In [5]:
# Nenhuma proteção contra nomes de campo errados, campos ausentes ou
# campos extras
car3 = {
    "colr": "green",
    "automatic": False,
    "windshield": "broken",
}
car3

{'colr': 'green', 'automatic': False, 'windshield': 'broken'}

### `tuple`: Grupo de Objetos Imutáveis

As tuplas do Python são uma estrutura de dados simples para agrupar objetos arbitrários. Tuplas são imutáveis - elas não podem ser modificadas depois de criadas.

Sob o ponto de vista da performance, tuplas ocupam um pouco menos memória do que as listas em CPython, e eles também são mais rápidos para construir.

Como você pode ver na desmontagem abaixo, construir uma tupla leva a um único opcode LOAD_CONST, enquanto a construção de um objeto de lista com o mesmo conteúdo requer várias mais operações.

In [6]:
import dis  # noqa E402

dis.dis(compile("(23, 'a', 'b', 'c')", "", "eval"))

  1           0 LOAD_CONST               0 ((23, 'a', 'b', 'c'))
              2 RETURN_VALUE


In [7]:
dis.dis(compile("[23, 'a', 'b', 'c']", "", "eval"))

  1           0 BUILD_LIST               0
              2 LOAD_CONST               0 ((23, 'a', 'b', 'c'))
              4 LIST_EXTEND              1
              6 RETURN_VALUE


No entanto, você não deve colocar muita ênfase nessas diferenças. Na prática, a diferença de desempenho, na maioria das vezes, será insignificante. Tentar  espremer desempenho extra, de um programa, alternando as listas para tuplas, provavelmente será uma abordagem equivocada.

Uma possível desvantagem de tuplas simples é que os dados que você armazenam neles só podem ser retirados, acessando-o através de índices inteiros. Você não pode dar nomes para propriedades individuais armazenadas em uma tupla. Isso pode impactar a legibilidade de código.

Além disso, uma tupla é sempre uma estrutura ad-hoc: é difícil garantir que duas tuplas tenham o mesmo número de campos e as mesmas propriedades armazenadas nelas.

Isso facilita a introdução de bugs difíceis de perceber, como misturar a ordem de campo. Portanto, eu recomendaria que você mantenha o número de campos armazenados em uma tupla o mais baixo possível.

In [8]:
# Campos: color, mileage, automatic
car1 = ("red", 3812.4, True)
car2 = ("blue", 40231.0, False)

In [9]:
# car1.__repr__()
car1

('red', 3812.4, True)

In [10]:
# car2.__repr__()
car2

('blue', 40231.0, False)

In [11]:
import traceback  # noqa E402

try:
    # Tuplas são imutáveis
    car2[1] = 12
except TypeError:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\josen\AppData\Local\Temp/ipykernel_75456/2323499011.py", line 5, in <module>
    car2[1] = 12
TypeError: 'tuple' object does not support item assignment


In [12]:
# Nenhuma proteção contra nomes de campo errados, campos ausentes ou
# campos extras
car3 = (3431.5, "green", True, "silver")
car3

(3431.5, 'green', True, 'silver')

### Escreva uma classe personalizada: mais trabalho, mais controle

Uma classe permite que você defina um modelo reutilizável para objetos de dados, para garantir que cada objeto forneça o mesmo conjunto de campos.

Usar classes regulares como tipos de dados registro é viável, mas também é necessário trabalho manual para obter as conveniências de outras implementações.Por exemplo, adicionar novos campos ao construtor `__init__` é verboso e leva tempo.

Além disso, a representação padrão de string para objetos instanciados a partir de classes personalizadas não é muito útil. Para corrigir isso, você pode ter que adicionar seu próprio método `__repr__` que, geralmente, é bastante detalhado e deve ser atualizado toda vez que você adicionar um novo campo.

Os campos das classes são mutáveis e novos campos podem ser adicionados livremente, quer você goste ou não. É possível ter mais controle de acesso e criar campos somente leitura usando o decorador `@property` mas, mais uma vez, isso requer escrever mais código de cola.

Escrever uma classe personalizada é uma ótima opção sempre que você quiser  adicionar lógica e comportamento de negócios aos seus objetos de registro usando métodos. No entanto, isso significa que esses objetos não são mais, tecnicamente, objetos de dados simples.

In [13]:
class Car:
    def __init__(self, color, mileage, automatic):
        self.color = color
        self.mileage = mileage
        self.automatic = automatic


car1 = Car("red", 3812.4, True)
car2 = Car("blue", 40231.0, False)

In [14]:
# Obtendo a propriedade mileage
car2.mileage

40231.0

In [15]:
# Classes são mutáveis
car2.mileage = 12
car2.windshield = "broken"

In [16]:
# A representação de string não é muito útil
# (o método __repr__ deve ser sobreescrito manualmente):
car2

In [17]:
def repr(self):
    return f"Car({self.color}, {self.mileage}, {self.automatic})"


Car.__repr__ = repr

car2

Car(blue, 12, False)

### `dataclasses.dataclass`: Python 3.7+ Data Classes

Data classes estão disponíveis a partir do Python 3.7. Elas fornecem uma excelente alternativa para definir suas próprias classes de armazenamento de dados do zero.

Ao escrever uma data class, em vez de uma classe Python normal, suas instâncias de objeto obtêm alguns recursos úteis por padrão, que economizarão algum trabalho de digitação e implementação manual

- A sintaxe para definir variáveis de instância é mais curta, já que você não precisa implementar o método `.__init__ ()`.
- As instâncias de sua classe de dados obtêm, automaticamente, uma representação de string útil, por meio de um método `.__repr__()` gerado automaticamente.
- As variáveis de instância aceitam anotações de tipo, fazendo ocm que a classe tenha um certo grau de auto-documentação. Tenha em mente que anotações de tipo são apenas sugestões, que não são aplicadas sem uma ferramenta de verificação de tipo separada.

As data class são criadas usando o decorador `@dataclass`, como você verá no exemplo abaixo:

In [18]:
from dataclasses import dataclass  # noqa E402


@dataclass
class Car:
    color: str
    mileage: float
    automatic: bool


car1 = Car("red", 3812.4, True)

In [19]:
# car1.__repr__()
car1

Car(color='red', mileage=3812.4, automatic=True)

In [20]:
# Acessando campos
car1.mileage

3812.4

In [21]:
# Campos são mutáveis
car1.mileage = 12
car1.windshield = "broken"

In [22]:
# Anotações de tipo não são verificadas sem uma ferramenta de verificação de
# tipo, como o mypy
Car("red", "NOT_A_FLOAT", 99)

Car(color='red', mileage='NOT_A_FLOAT', automatic=99)

### `collections.namedtuple`: Objetos de dados convenientes

A classe `namedtuple`. disponível a partir do Python 2.6, fornece uma extensão da classe built-in `tuple`. Similar a definir uma classe customizada, usar a `namedtuple` permite que defina um modelo reutilizável para seus registros, garantindo que os nomes de campos certos sejam utilizados.

Objetos `namedtuple` são imutáveis, da mesma forma que as tuplas normais. Isso significa que você não pode adicionar novos campos, ou modificar campos existentes depois que a instância da Tupla é criada.

Portanto, objetos `namedtuple`, como o próprio nome já diz, são tuplas **nomeadas**, ou seja, você não está preso a índices numéricos, nem precisa de gambiarras, como definir constantes inteiras pra usar como mnemônicos para os índices.

**Atenção:**

> Objetos `namedtuple` são implementados como objetos Python normais,
> internamente. POrém, quando comparados a objetos normais, em relação ao
> consumo de memória, eles são melhores que classes normais, sendo tão
> eficientes quanto tuplas normais.

***O texto diz isso mas, como podemos ver pelo resultado abaixo, é melhor analisar cada caso.***

In [23]:
from collections import namedtuple  # noqa E402
from sys import getsizeof  # noqa E402


class PointClass:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __repr__(self):
        return f"PointClass({self.x}, {self.y}, {self.z})"


p1 = namedtuple("Point", "x y z")(1, 2, 3)
p2 = (1, 2, 3)
p3 = PointClass(1, 2, 3)

print(getsizeof(p1))
print(getsizeof(p2))
print(getsizeof(p3))

64
64
48


In [24]:
p1

Point(x=1, y=2, z=3)

In [25]:
p2

(1, 2, 3)

In [26]:
p3

PointClass(1, 2, 3)


Mudar de esturuturas de dados ad-hoc, como dicionários e tuplas normais, para `namedtuples` ajuda o desenvolvedor a comunicar melhor suas intenções, tornando o código mais legível, simples e compreensível, além de fornecer uma estrutura de dados mais robusta e segura. Isso também facilita o trabalho em equipe, visto que o código se torna, até certo ponto, autodocumentado.

In [27]:
from typing import NamedTuple  # noqa E402


class Car(NamedTuple):
    color: str
    mileage: float
    automatic: bool


car1 = Car("red", 3812.4, True)

In [28]:
# car1.__repr__()
car1

Car(color='red', mileage=3812.4, automatic=True)

In [29]:
# Acessando campos
car1.mileage

3812.4

In [30]:
import traceback  # noqa E402

try:
    # campos são imutáveis
    car1.mileage = 12
except AttributeError:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\josen\AppData\Local\Temp/ipykernel_75456/3100363015.py", line 5, in <module>
    car1.mileage = 12
AttributeError: can't set attribute


In [31]:
import traceback  # noqa E402

try:
    # Não é possível adicionar campos
    car1.windshield = "broken"
except AttributeError:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\josen\AppData\Local\Temp/ipykernel_75456/3210627203.py", line 5, in <module>
    car1.windshield = "broken"
AttributeError: 'Car' object has no attribute 'windshield'


In [32]:
# Anotações de tipo não são verificadas sem uma ferramenta de verificação de
# tipo, como o mypy
Car("red", "NOT_A_FLOAT", 99)

Car(color='red', mileage='NOT_A_FLOAT', automatic=99)

### `typing.NamedTuple`: Namedtuples Melhoradas

Adicionado no Python 3.6, `typing.NamedTuple` é a irmã mais nova da classe
`namedtuple` no módulo de coleções. Ela é muito semelhante à `namedtuple`, com
a principal diferença que tem uma sintaxe atualizada para definir novos tipos
de registro e suporte para type hints.

Note que as anotações de tipo não são verificadas em tempo de execução ou
"compilação, e precisam de uma ferramenta de verificação de tipo separada, como
o Mypy. Mas, mesmo sem o suporte à ferramenta, elas podem dar dicas úteis aos  
outros programadores (ou serem terrivelmente confusas, se as type hints ficarem
desatualizadas):

In [33]:
from typing import NamedTuple  # noqa E402


class Car(NamedTuple):
    color: str
    mileage: float
    automatic: bool


car1 = Car("red", 3812.4, True)

In [34]:
# car1.__repr__()
car1

Car(color='red', mileage=3812.4, automatic=True)

In [35]:
# Acessando campos
car1.mileage

3812.4

In [36]:
import traceback  # noqa E402


try:
    # Campos são imutáveis
    car1.mileage = 12
except AttributeError:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\josen\AppData\Local\Temp/ipykernel_75456/4021539728.py", line 6, in <module>
    car1.mileage = 12
AttributeError: can't set attribute


In [37]:
import traceback  # noqa E402


try:
    # Campos são imutáveis
    car1.windshield = "broken"
except AttributeError:
    traceback.print_exc()

Traceback (most recent call last):
  File "C:\Users\josen\AppData\Local\Temp/ipykernel_75456/1302433663.py", line 6, in <module>
    car1.windshield = "broken"
AttributeError: 'Car' object has no attribute 'windshield'


In [38]:
# Anotações de tipo não são verificadas sem uma ferramenta de verificação de
# tipo, como o mypy
Car("red", "NOT_A_FLOAT", 99)

Car(color='red', mileage='NOT_A_FLOAT', automatic=99)

### `struct.Struct`: Structs C Serializados

A classe `struct.Struct` converte valores Python e structs C serializadas em objetos Python. POr exemplo, ela pode ser usada para manipular dados minários armazenados em arquivos ou recebidos pela rede.

As structs são definidas usando uma mini linguagem, com base em strings de formato, que permite definir o arranjo de vários tipos de dados C como char, int e longos, assim como suas variantes não sinalizadas.

Structs serializados raramente são usados para representar objetos de dados destinados a serem tratados apenas dentro do código Python. Eles são destinados principalmente como um formato de troca de dados, em vez de como uma maneira de manter dados na memória que é usada apenas pelo código Python.

Em alguns casos, enpacotar dados primitivos em struct pode usar menos memória do que mantê-lo em outros tipos de dados. No entanto, na maioria dos casos, seria uma otimização bastante avançada (e provavelmente desnecessária):

In [39]:
from struct import Struct  # noqa E402


MyStruct = Struct("i?f")
data = MyStruct.pack(23, False, 42.0)

In [40]:
# Tudo o que você recebe é um blob de dados:
data

b'\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00(B'

In [41]:
# Blobs de dados podem ser descompactados novamente:
MyStruct.unpack(data)

(23, False, 42.0)

### `types.SimpleNamespace`: Acesso elegante a atributos

Aqui está uma escolha mais ligeiramente obscura para implementar objetos de dados no Python: `types.SimpleNamespace`. Esta classe foi adicionada no Python 3.3 e fornece acesso a atributos ao seu namespace.

Isso significa que os objetos `SimpleNamespace` expõem todas as suas chaves como atributos de classe. Você pode usar a sintaxe de ponto (obj.key) ao invés  da sintaxe de indexação (obj['key']). Todas as instâncias também incluem uma implementação significativa do `.__repr__()`.

Como seu nome proclama, o `SimpleNamespace` é simples! É basicamente um dicionário que permite acesso como atributo e imprime bem. Os atributos podem ser adicionados, modificados e excluídos livremente.

In [42]:
from types import SimpleNamespace  # noqa E402

car1 = SimpleNamespace(color="red", mileage=3812.4, automatic=True)

In [43]:
# car1.__repr__()
car1

namespace(color='red', mileage=3812.4, automatic=True)

In [44]:
# Instâncias suportam acesso ao atributo e são mutáveis:
car1.mileage = 12
car1.windshield = "broken"
del car1.automatic
car1

namespace(color='red', mileage=12, windshield='broken')

### Registros, Estuturas e Objetos de Transferência de Dados: Resumo

Como você viu, há uma série de opções diferentes para implementar registros ou objetos de dados. Qual tipo você deve usar para objetos de dados no Python?Geralmente sua decisão dependerá do seu caso de uso:

- Se você tiver apenas alguns campos, uma tupla simples pode ser uma boa escolha se a ordem de campo for fácil de lembrar ou os nomes de campo são supérfluos. Por exemplo, pense em um ponto (x, y, z) em espaço tridimensional.
- Se você precisar de campos imutáveis, `tuples`, `collections.namedtuple` e `typing.NamedTuple` são boas opções.
- Se você precisar travar os nomes dos campos, para evitar erros de digitação, então `collections.namedtuple` e `typing.NamedTuple` podem ser usados.
- Se você quiser manter as coisas simples, então um `dict` pode ser uma boa escolha, devido à sintaxe que se assemelha a JSON.
- Se você precisar de controle total sobre sua estrutura de dados, então é hora de escrever uma classe customizada, com propriedades (`@property` para getters, `@xxx.setter` para setters).
- Se você precisar adicionar comportamento (métodos) ao objeto, então você deve escrever uma classe customizada, seja do zero ou usando o decorador `@dataclass`, ou estendendo `collections.namedtuple` or `typing.NamedTuple`.
- Se você precisar compactar dados, para serializá-lo no disco ou para enviá-lo pela rede, então é hora de ler sobre `struct.Struct`, porque este é um grande caso de uso para ele!

Se você estiver procurando por uma opção de padrão segura para implementar um registro, struct ou objeto de dados simples, em Python, a recomendação geral seria usar `collections.namedtuple`, em Python 2.x, e seu irmão mais novo, `typing.NamedTuple`, no Python 3.x.